# Purpose of the Notebook:

This notebook is all about prediction.
For this purpose, two different regressors are taken into account and their performance is compared.
Furthermore, these regressors are saved for further usage.


# Forecasting

In [1]:
# imports
import pandas as pd
import X3_Forecasting as forecast
import orga_functions as org
import pickle

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import d2_absolute_error_score

In [2]:
# import the regressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 

In [3]:
# read in the data set
#df = pd.read_csv(org.path("02_AirQuality_processed.csv"), sep=';', index_col='date')
df = pd.read_csv(org.path("03_AirQuality_normalized.csv"), sep=';', index_col="date")

> We can distinguish between the data in its original form and the normalised form.
Normalising the AH values leads to the problem that the results can no longer be interpreted by humans.
As an temporary solution, all values except AH were normalised.

In [4]:
df

,co_gt,pt08_s1_co,nmhc_gt,c6h6_gt,pt08_s2_nmhc,nox_gt,pt08_s3_nox,no2_gt,pt08_s4_no2,pt08_s5_o3,t,rh,ah
date,,,,,,,,,,,,,
2004-03-10 18:00:00,0.454545,0.688224,0.225197,0.409722,0.311975,0.263242,0.649558,0.509174,0.616822,0.557692,0.308585,0.499371,0.7578
2004-03-10 19:00:00,0.345455,0.622587,0.165354,0.322917,0.216387,0.162119,0.753982,0.412844,0.533956,0.395055,0.301624,0.484277,0.7255
2004-03-10 20:00:00,0.381818,0.728764,0.127559,0.309028,0.199580,0.207063,0.723894,0.513761,0.531464,0.451099,0.269142,0.563522,0.7502
2004-03-10 21:00:00,0.381818,0.703668,0.114961,0.315972,0.209034,0.272873,0.681416,0.550459,0.549533,0.521978,0.248260,0.638994,0.7867
2004-03-10 22:00:00,0.272727,0.603282,0.069291,0.222222,0.091387,0.207063,0.781416,0.522936,0.490966,0.470879,0.252900,0.633962,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-02-06 19:00:00,0.272727,0.326255,0.333240,0.152778,0.214894,0.361156,0.503540,0.747706,0.107788,0.286264,0.132251,0.362264,0.3584
2005-02-06 20:00:00,0.309091,0.342664,0.333240,0.180556,0.032563,0.401284,0.471681,0.811927,0.118380,0.331868,0.127610,0.342138,0.3385
2005-02-06 21:00:00,0.236364,0.280888,0.333240,0.125000,0.214894,0.306581,0.544248,0.674312,0.064174,0.265934,0.127610,0.329560,0.3286


## Features

In [5]:
# list of features which will be used for the algorithm - based on 05_Feature_selection
features = ["t", "ah"]

## Target

In [6]:
target = 'ah_target'

In [7]:
# shift of the absolute humidty values by 6 hrs
target = 'ah_target'
df[target] = df.ah.shift(periods=-6)

In [8]:
df.head(8)

,co_gt,pt08_s1_co,nmhc_gt,c6h6_gt,pt08_s2_nmhc,nox_gt,pt08_s3_nox,no2_gt,pt08_s4_no2,pt08_s5_o3,t,rh,ah,ah_target
date,,,,,,,,,,,,,,
2004-03-10 18:00:00,0.454545,0.688224,0.225197,0.409722,0.311975,0.263242,0.649558,0.509174,0.616822,0.557692,0.308585,0.499371,0.7578,0.7603
2004-03-10 19:00:00,0.345455,0.622587,0.165354,0.322917,0.216387,0.162119,0.753982,0.412844,0.533956,0.395055,0.301624,0.484277,0.7255,0.7702
2004-03-10 20:00:00,0.381818,0.728764,0.127559,0.309028,0.199580,0.207063,0.723894,0.513761,0.531464,0.451099,0.269142,0.563522,0.7502,0.7648
2004-03-10 21:00:00,0.381818,0.703668,0.114961,0.315972,0.209034,0.272873,0.681416,0.550459,0.549533,0.521978,0.248260,0.638994,0.7867,0.7517
2004-03-10 22:00:00,0.272727,0.603282,0.069291,0.222222,0.091387,0.207063,0.781416,0.522936,0.490966,0.470879,0.252900,0.633962,0.7888,0.7465
2004-03-10 23:00:00,0.200000,0.530888,0.048819,0.159722,0.001050,0.139647,0.898230,0.431193,0.430530,0.382418,0.252900,0.628931,0.7848,0.7366
2004-03-11 00:00:00,0.200000,0.519305,0.037795,0.121528,0.214894,0.096308,0.466162,0.344037,0.393146,0.263736,0.255220,0.598742,0.7603,0.7353
2004-03-11 01:00:00,0.163636,0.472008,0.037795,0.111111,0.214894,0.096308,0.466162,0.339450,0.393146,0.262088,0.241299,0.638994,0.7702,0.7417


> As we can see, the ah_target value from the first line equals to the ah value from the seventh line indicating that the shift was successful.

In [9]:
df.shape

(7998, 14)

## Limitation of the ML_DataFrame

In [10]:
# drop rows where target is unknown
df.dropna(subset=[target], inplace=True)

> due to the shift last 6 rows with NaN ah_target are dropped

In [11]:
df

,co_gt,pt08_s1_co,nmhc_gt,c6h6_gt,pt08_s2_nmhc,nox_gt,pt08_s3_nox,no2_gt,pt08_s4_no2,pt08_s5_o3,t,rh,ah,ah_target
date,,,,,,,,,,,,,,
2004-03-10 18:00:00,0.454545,0.688224,0.225197,0.409722,0.311975,0.263242,0.649558,0.509174,0.616822,0.557692,0.308585,0.499371,0.7578,0.7603
2004-03-10 19:00:00,0.345455,0.622587,0.165354,0.322917,0.216387,0.162119,0.753982,0.412844,0.533956,0.395055,0.301624,0.484277,0.7255,0.7702
2004-03-10 20:00:00,0.381818,0.728764,0.127559,0.309028,0.199580,0.207063,0.723894,0.513761,0.531464,0.451099,0.269142,0.563522,0.7502,0.7648
2004-03-10 21:00:00,0.381818,0.703668,0.114961,0.315972,0.209034,0.272873,0.681416,0.550459,0.549533,0.521978,0.248260,0.638994,0.7867,0.7517
2004-03-10 22:00:00,0.272727,0.603282,0.069291,0.222222,0.091387,0.207063,0.781416,0.522936,0.490966,0.470879,0.252900,0.633962,0.7888,0.7465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-02-06 13:00:00,0.200000,0.254826,0.333240,0.118056,0.214894,0.298555,0.583186,0.610092,0.054829,0.167582,0.250580,0.179874,0.3105,0.3584
2005-02-06 14:00:00,0.163636,0.213320,0.333240,0.069444,0.214894,0.200642,0.671681,0.449541,0.031776,0.091758,0.238979,0.211321,0.3320,0.3385
2005-02-06 15:00:00,0.127273,0.213320,0.333240,0.062500,0.214894,0.150883,0.713274,0.348624,0.033022,0.060440,0.232019,0.232704,0.3481,0.3286


In [12]:
df.shape

(7992, 14)

## Preparation Data

The data set is split into a training set and a test set

In [13]:
# import the split function
from sklearn.model_selection import train_test_split

# split
training, test = train_test_split(df, test_size=1000, shuffle=False, random_state=1999)

In [14]:
# classification of features and the target
X_train = training[features]
y_train = training[target]
x_test = test[features]
y_test = test[target]

> Both sets are subdivided into the features and the target, thus all other columns are not included any further

In [15]:
X_train

,t,ah
date,,
2004-03-10 18:00:00,0.308585,0.7578
2004-03-10 19:00:00,0.301624,0.7255
2004-03-10 20:00:00,0.269142,0.7502
2004-03-10 21:00:00,0.248260,0.7867
2004-03-10 22:00:00,0.252900,0.7888
...,...,...
2004-12-26 21:00:00,0.266821,1.1381
2004-12-26 22:00:00,0.266821,1.0741
2004-12-26 23:00:00,0.271462,1.0095


> As an example, nox was not defined as a feature. It can be seen here that the nox column is no longer a part of the dataframe.

## Models/Regressors

### Linear Regression

In [16]:
# The regressor gets trained 
clf_lin_reg = LinearRegression()
clf_lin_reg.fit(X_train, y_train)

LinearRegression()

In [17]:
# values are estimated by the regressor
pred_linr_y = clf_lin_reg.predict(x_test)

### Performance Check

In [18]:
# visual comparison of the actual and estimated values
check_linr = forecast.check_df(x_test, y_test, clf_lin_reg)
check_linr

,future_ah,predicted_ah
future_datetime,,
2004-12-27 08:00:00,0.9302,0.914418
2004-12-27 09:00:00,0.9113,0.887897
2004-12-27 10:00:00,0.9335,0.870580
2004-12-27 11:00:00,0.9261,0.841111
2004-12-27 12:00:00,0.9379,0.822809
...,...,...
2005-02-06 19:00:00,0.3584,0.375556
2005-02-06 20:00:00,0.3385,0.390809
2005-02-06 21:00:00,0.3286,0.402679


In [19]:
#mean_absolute_error of the predicted AH value compared to the actual future AH value
mean_absolute_error(y_test, pred_linr_y)

0.0563969743878939

In [20]:
# variance of the predicted AH value compared to the actual future AH value
d2_absolute_error_score(y_test, pred_linr_y)

0.6954328439949587

## Decision Tree

In [21]:
# The regressor gets trained
clf_tree = DecisionTreeRegressor(random_state = 0) 
clf_tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [22]:
# values are estimated by the regressor
pred_tree_y = clf_tree.predict(x_test)

### Performance Check

In [23]:
# visual comparison of the actual and estimated values
check_tree_df = forecast.check_df(x_test, y_test, clf_tree)
check_tree_df

,future_ah,predicted_ah
future_datetime,,
2004-12-27 08:00:00,0.9302,0.9206
2004-12-27 09:00:00,0.9113,1.0727
2004-12-27 10:00:00,0.9335,0.8063
2004-12-27 11:00:00,0.9261,0.7616
2004-12-27 12:00:00,0.9379,0.8225
...,...,...
2005-02-06 19:00:00,0.3584,0.4273
2005-02-06 20:00:00,0.3385,0.4799
2005-02-06 21:00:00,0.3286,0.5062


In [24]:
#mean_absolute_error of the prediction and the actual values
mean_absolute_error(y_test, pred_tree_y)

0.08367690000000001

In [25]:
# variance of the prediction and the actual values
d2_absolute_error_score(y_test, pred_tree_y)

0.5481098811962355

## Saving

### Save Training/Test Split

In [26]:
# save training dataset locally
new_path = org.path("00_training.csv")
training.to_csv(new_path, sep=';', index = True)

In [27]:
# save test dataset locally
new_path = org.path("00_testing.csv")
test.to_csv(new_path, sep=';', index = True)

In [28]:
# save the x_test dataset
x_test_path = org.path("00_x_test.csv")
x_test.to_csv(x_test_path, sep=';', index = True)

# save the y_test dataset
y_test_path = org.path("00_y_test.csv")
y_test.to_csv(y_test_path, sep=';', index = True)

# save the x_train dataset
x_train_path = org.path("00_x_train.csv")
X_train.to_csv(x_train_path, sep=';', index = True)

# save the y_train dataset
y_train_path = org.path("00_y_train.csv")
y_train.to_csv(y_train_path, sep=';', index = True)

### Save Model

In [29]:
# the trained LinearRegressor is saved as a pickle file
new_path = org.path("LinearRegression_ah_regressor.pkl")

with open(new_path, 'wb') as f:
    pickle.dump(clf_lin_reg, f)

In [30]:
# the trained DecisionTree is saved as a pickle file
new_path = org.path("DecisionTree_ah_regressor.pkl")

with open(new_path, 'wb') as f:
    pickle.dump(clf_tree, f)

> These regressors are stored to be used later for unit tests and also in reality.